In [ ]:
!pip -q install segmentation-models-pytorch==0.3.3 pillow opencv-python matplotlib torch torchvision torchmetrics
import torch, torch.nn as nn, cv2, numpy as np, matplotlib.pyplot as plt,  matplotlib
from PIL import Image
import segmentation_models_pytorch as smp
from pathlib import Path, PurePosixPath
matplotlib.rcParams["figure.dpi"]=120
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", DEVICE)

In [ ]:
img_2010 = Image.open('orlando2010.png').convert('RGB')
img_2023 = Image.open('orlando2023.png').convert('RGB')

plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.title('2010'); plt.imshow(img_2010); plt.axis('off')
plt.subplot(1,2,2); plt.title('2023'); plt.imshow(img_2023); plt.axis('off'); plt.show()

In [ ]:
import timm

class UNetBuildings(nn.Module):
    def __init__(self, weights_url):
        super().__init__()
        self.net = smp.Unet(encoder_name="efficientnet-b0", encoder_weights=None,
                            classes=1, activation='sigmoid')
        self.net.load_state_dict(torch.hub.load_state_dict_from_url(weights_url, map_location='cpu'))
    def forward(self,x): return self.net(x)

b_weights = "https://huggingface.co/FLAME/unet-eff-b0_buildings/resolve/main/pytorch_model.bin"
b_model   = UNetBuildings(b_weights).to(DEVICE).eval()

def infer_mask(model, pil_img, thresh=0.35):
    resize = (512,512)
    img = pil_img.resize(resize)
    arr = torch.tensor(np.array(img)/255.).permute(2,0,1).unsqueeze(0).float().to(DEVICE)
    with torch.no_grad():
        pred = model(arr)[0,0].cpu().numpy()
    mask = (pred>thresh).astype(np.uint8)
    mask = cv2.resize(mask, pil_img.size, interpolation=cv2.INTER_NEAREST)
    return mask

In [ ]:
import torchvision
r_model = torchvision.models.segmentation.deeplabv3_resnet50(weights="DEFAULT").to(DEVICE).eval()
ROAD_LABELS = [0, 1, 2]  # Mapillary classes for road-like surfaces

def infer_roads(pil_img, conf=0.50):
    resize = (520,520)
    img = pil_img.resize(resize)
    tens = torchvision.transforms.functional.to_tensor(img).unsqueeze(0).to(DEVICE)
    with torch.no_grad(): out = r_model(tens)['out'][0].softmax(0).cpu()
    road_mask = np.zeros(out.shape[1:], dtype=np.uint8)
    for lbl in ROAD_LABELS:
        road_mask |= (out[lbl]>conf).numpy().astype(np.uint8)
    road_mask = cv2.resize(road_mask, pil_img.size, interpolation=cv2.INTER_NEAREST)
    return road_mask

In [ ]:
# ----- run both detectors -----
b10, b23 = infer_mask(b_model, img_2010), infer_mask(b_model, img_2023)
r10, r23 = infer_roads(img_2010),        infer_roads(img_2023)

# ----- new-infrastructure masks -----
new_build = (b23 & ~b10).astype(np.uint8)
new_road  = (r23 & ~r10).astype(np.uint8)

# ----- overlay on 2023 image -----
vis = np.array(img_2023).copy()
vis[new_build==1] = [255,0,0]         # red buildings
vis[new_road==1]  = [255,255,0]       # yellow roads

plt.figure(figsize=(6,6)); plt.imshow(vis); plt.axis('off')
plt.title('Neural Detection Overlay (2023)'); plt.show()

In [ ]:
def pct(mask): return 100*mask.sum()/mask.size

report = {
    'building_2010_%': pct(b10),
    'building_2023_%': pct(b23),
    'road_2010_%':     pct(r10),
    'road_2023_%':     pct(r23),
}
report['building_change_%'] = report['building_2023_%']-report['building_2010_%']
report['road_change_%']     = report['road_2023_%']-report['road_2010_%']

def text_report(d):
    more_b = d['building_change_%']>0
    more_r = d['road_change_%']>0
    summary = ("substantial expansion" if d['building_change_%']>2 or d['road_change_%']>2 
               else "moderate changes")
    bullets = [
      f"Built-up footprint {'increased' if more_b else 'decreased'} by "
      f"{abs(d['building_change_%']):.1f}% (from {d['building_2010_%']:.1f}% to {d['building_2023_%']:.1f}%).",
      f"Paved road/taxiway coverage {'expanded' if more_r else 'reduced'} by "
      f"{abs(d['road_change_%']):.1f}% (from {d['road_2010_%']:.1f}% to {d['road_2023_%']:.1f}%)."
    ]
    return f"""
ORLANDO AIRPORT – NEURAL NETWORK CHANGE REPORT
Generated {datetime.date.today().isoformat()}

Executive Summary
-----------------
Automated deep-learning segmentation reveals **{summary}** between 2010 and 2023.

Key Findings
------------
• {bullets[0]}
• {bullets[1]}

Method
------
U-Net (EfficientNet-b0) for building footprints, DeepLabV3-R50 for roads/taxiways.
Pixel-wise difference yields new-infrastructure masks visualised in overlay.

Limitations
-----------
Model weights are generic; fine-tuning on local imagery would improve precision.
Pixel-percent metrics assume equal scale and alignment of source images.
"""
print(text_report(report))